In [1]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/RNN/rnn_ext/rnn_ext_model.ipynb'

In [2]:
%run {RUN_DIR}

In [3]:
class Autoencoder(RnnExtModel):
    def __init__(self, name, dataset, hconfigs, show_maps = False, l2_decay = 0, \
                l1_decay = 0, dump_structure = False, fix_encoder = False):
        self.fix_encoder = fix_encoder
#         self.pm_encoder, self.pm_decoder, self.pm_hiddens = None,None,None
        # encoder학습 동결시킬지 (파라미터 가변 or 고정)
        super(Autoencoder, self).__init__(name, dataset, hconfigs,\
                                         show_maps, l2_decay, l1_decay, dump_structure)
        

In [ ]:
def autoencoder_init_parameters(self, hconfigs):
    econf = hconfigs['encoder']
    dconf = hconfigs['decoder']
    hconf = hconfigs['supervised']
    
    in_shape = self.dataset.input_shape
    
    pme, code_shape = self.build_subnet(econf, in_shape)
    pmd, represent_shape = self.build_subnet(dconf, code_shape)
    pmh, hidden_shape = self.build_subnet(hconf, code_shape)
    
    self.econfigs, self.dconfigs, self.hconfigs = econf, dconf, hconf
    
    self.pm_encoder, self.pm_decoder, self.pm_hiddens = pme, pmd, pmh
    
    output_cnt = int(np.prod(self.dataset.output_shape))
    self.seqout=False
    
    if len(hconf) > 0 and get_layer_type(hconf[-1]) in ['rnn','lstm']:
        if get_conf_param(hconf[-1], 'outseq',True):
            self.seqout = True
            hidden_shape = hidden_shape[1:]
            output_cnt = int(np.prod(self.dataset.output_shape[1:]))
            
    self.pm_output, _ = self.alloc_layer_param(hidden_shape, output_cnt)
    
    
def autoencoder_build_subnet(self, hconfigs, prev_shape):
    pms = []
    
    for hconfig in hconfigs:
        
        pm, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        pms.append(pm)
        
    return pms, prev_shape


Autoencoder.build_subnet = autoencoder_build_subnet
Autoencoder.init_parameters = autoencoder_init_parameters

In [ ]:
def autoencoder_autoencode(self,epoch_count = 10, batch_size=10,\
                           learning_rate=0.001, report=0):
    self.learning_rate = learning_rate
    batch_count = self.dataset.autoencode_count // batch_size
    time1 = time2 = int(time.time())

    self.Auto_model, optim, loss = self.Auto_get_torch_params()
    self.Auto_model.apply(self.weights_init) 
    
    if report !=0:
        print('Model {}autoencode started:'.format(self.name))
        
    for epoch in range(epoch_count):
        costs =[]
        accs =[]
        self.dataset.shuffle_train_data(batch_size * batch_count)
        for n in range(batch_count):
            trX = self.dataset.get_autoencode_data(batch_size, n)   
            trX=torch.from_numpy(trX).float()
            optim.zero_grad()
    
            output=self.Auto_model.forward(trX)
#             self.output = output
#             self.trX = trX
            cost=loss(output.view(batch_size,1,28,28), trX.view(batch_size,1,28,28))
            acc = 1-(torch.sqrt(cost)/torch.mean(trX))

            cost.backward()

            optim.step()
#             cost, acc = self.autoencoder_torch_step(trX, optim, loss)
            
            costs.append(cost)
            accs.append(acc)
            
        if report > 0 and(epoch +1) % report == 0:
            self.T_costs=costs
            self.T_accs =accs
            acc = torch.mean(torch.FloatTensor(accs))
            time3=int(time.time())
            tm1, tm2 = time3 - time2, time3 - time1
            
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            time2 = time3
            
    tm_total = int(time.time()) - time1
    if report != 0:
        print('Model {} autoencode ended in {} secs :'.format(self.name, tm_total))
        
    self.layers.append(self.fix_encoder)
    self.layers.append(self.Auto_model.encoder_layers)
    self.layers.extend(self.pm_hiddens[0])
    
Autoencoder.autoencode = autoencoder_autoencode

In [83]:
class Autoencoder_layers(nn.Module):
    def __init__(self, *args):
        nn.Module.__init__(self)
        self.pm_encoder, self.pm_decoder, self.pm_hiddens = args
        
        self.encoder_layers = nn.Sequential(*self.pm_encoder[0])
        self.decoder_layers = nn.Sequential(*self.pm_decoder[0])
        self.encoder_layers.add_module(name='ReLU',module=nn.LeakyReLU())
#         self.encoder_layers.add_module(name='sigmoid',module=nn.Sigmoid())
        self.decoder_layers.add_module(name='relu',module=nn.ReLU())
    
#         self.decoder_layers.add_module(name='Sigmoid2',module=nn.Sigmoid())
    
    def forward(self, x):
        
        self.encoder_forward = self.encoder_layers(x)
        self.decoder_forward = self.decoder_layers(self.encoder_forward)
        
        return self.decoder_forward

In [ ]:
def Auto_get_torch_params(self):
    
    self.auto_model = Autoencoder_layers(self.pm_encoder, self.pm_decoder, self.pm_hiddens).to(DEVICE)
    optim=torch.optim.Adam(self.auto_model.parameters(),lr=self.learning_rate,\
                             betas=(0.9, 0.999),eps=1e-08)
    loss = nn.MSELoss()
    return self.auto_model, optim, loss

Autoencoder.Auto_get_torch_params=Auto_get_torch_params

In [ ]:
def autoencoder_torch_step(self, trX, optim, loss):
    optim.zero_grad()
    
    output=self.Auto_model.forward(trX)
    
    cost=loss(trX, output)
    acc = 1-(torch.sqrt(cost)/torch.mean(trX))
    
    cost.backward()
    
    optim.step()
    
    return cost, acc

Autoencoder.autoencoder_torch_step = autoencoder_torch_step

In [ ]:
#지도 학습 
def autoencoder_forward_neuralnet(self, x ):
    hidden = x
    
    aux_encoder =[]
    
    for n, hconfig in enumerate(self.econfigs):
        hidden,aux = self.forward_layer(hidden, hconfig, self.pm_encoder[n])
        aux_encoder.append(aux)
        
    output, aux_layers = super(Autoencoder, self).forward_neuralnet(hidden)
    
    return output, [aux_encoder, aux_layers]


Autoencoder.forward_neuralnet = autoencoder_forward_neuralnet

In [ ]:
class Net(nn.Module):
    def __init__(self, args):
        super(Net, self).__init__()
        print(args)
        self.args = args
        if isinstance(args[0], bool):
            self.fix_encoder, self.encoder, self.fc = args
        else:
            layers = sum(args,[])
            self.layer1 = nn.Sequential(*layers)
#             self.layer1.add_module(name='relu',module=nn.ReLU())
            
    def forward(self, x):
        if isinstance(self.args[0], bool):
            if self.fix_encoder:
                 with torch.no_grad():
                    out = self.encoder(x)
            else:
                out = self.encoder(x)
                out = self.fc(out)
        else:
            out=self.layer1(x)
            
        return out

In [ ]:
# def test_torch(self,batch_size):
#     with torch.no_grad():
# #         print(self.model.parameters)
#         test_X,test_y=self.dataset.get_test_data(batch_size)
#         test_X=torch.from_numpy(test_X).float()
#         test_y=torch.from_numpy(test_y).float()
# #         print(test_X.shape,test_y.shape,'테스트')
#         time1 = int(time.time())        
#         acc = self.eval_accuracy(test_X, test_y)
#         time2 = int(time.time())
#         self.dataset.test_prt_result(self.name, acc, time2-time1)

# RnnExtModel.test = test_torch

In [ ]:
def test_torch(self,batch_size):
    with torch.no_grad():
        test_X,test_y=self.dataset.get_test_data(batch_size)
        time1 = int(time.time())        
        acc = self.eval_accuracy(test_X, test_y)
        time2 = int(time.time())
        self.dataset.test_prt_result(self.name, acc, time2-time1)

RnnExtModel.test = test_torch

In [ ]:
def mlp_model_visualize_torch(self, num):
    
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    
    copy = self.Auto_model(deX)
    copy=copy.detach().numpy()
    est = self.get_estimate(deX)
    
    self.dataset.autoencode_visualize(deX, copy, est, deY)

Autoencoder.visualize = mlp_model_visualize_torch

In [ ]:
def mlp_model_visualize_torch(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)
    self.dataset.visualize(deX, est, deY)

RnnExtModel.visualize = mlp_model_visualize_torch

In [ ]:
# def autoencoder_visualize(self, num):
#     print('Model {} Visualization'.format(self.name))
#     deX, deY = self.dataset.get_visualize_data(num)
#     est = self.get_estimate(deX)
#     self.dataset.autoencode_visualize(deX, copy, est, deY)
    
# Autoencoder.visualize = autoencoder_visualize